# DATA PREPARATION STARTING FROM DATAFRAMES

In [1]:
import itertools
import pathlib
import pickle as pkl

import numpy as np
import pandas as pd
# import torch

In [2]:
from dataset_prep import data_prep, data_prep_nosave

In [3]:
sim_folder = "/mnt/c/Users/nikko/OneDrive/Documents/Uni/magistrale/tesi/simulations"
datestamp = "20241108-115933"

data_prep(sim_folder, datestamp, 1e-5, 1000, 30, 2, f"prep{datestamp}")

In [108]:
datestamp = "20241108-110804"
sim_folder = "/mnt/c/Users/nikko/OneDrive/Documents/Uni/magistrale/tesi/simulations"

#Directory defining
sim_folder = pathlib.Path(sim_folder)
datafolder = sim_folder / datestamp / "data"

In [109]:
path_list = []
for path in datafolder.iterdir():
    if path.is_dir():
        num = path.name[3:]
        df_path = path / f"{datestamp}_run{num}.txt"
        path_list.append(df_path)

In [110]:
len(path_list)

1

In [111]:
df_list = []
for path in path_list:
    df_list.append(pd.read_csv(path))

In [112]:
timestep = 1e-5
if len(np.unique(df_list[0].Time)) != max(df_list[0].Time):
    timestep = timestep*(max(df_list[0].Time)-1)/(len(np.unique(df_list[0].Time))-1)

In [113]:
timestep = 1e-5
times = np.unique(df_list[0].Time)
downsampling_factor = times[1] - times[0]
timestep *= downsampling_factor*1000

In [114]:
timestep

np.float64(0.01)

## Velocity calc.

I am dropping the first instant for velocity is obv. NaN. 

In [134]:
L = 30
R = 2

In [140]:
for df in df_list:
    df.sort_values(by=["N","Time"],inplace=True)
    df[["vx","vy"]] = df.groupby("N")[["xpos","ypos"]].diff()/timestep #velocity calculation
    df[["ax","ay"]] = df.groupby("N")[["vx","vy"]].diff()/timestep #acceleration calculation
    df.dropna(inplace=True)
    df["Time"]=df["Time"]-(2*downsampling_factor)

In [116]:
df_list[0].xpos[20] - df_list[0].xpos[30]

np.float64(0.12198135036662272)

In [117]:
6.610125 - 6.602317

0.007807999999999815

In [118]:
a = df_list[0][df_list[0]["vx"]==df_list[0]["vx"].max()]

In [119]:
a

,N,Time,xpos,ypos,orientation,fx,fy,torque,vx,vy,ax,ay
6291,2,628,13.021874,3.046527,2.555829,-4.659346,-3.300597,0.0,2997.086773,-2.541743,301006.905408,-996.716646


In [131]:
a[a["vx"]>((30-2)/timestep)]

,N,Time,xpos,ypos,orientation,fx,fy,torque,vx,vy,ax,ay
6291,2,628,13.021874,3.046527,2.555829,-4.659346,-3.300597,0.0,2997.086773,-2.541743,301006.905408,-996.716646


In [132]:
a["vx"]-(30/timestep)

6291   -2.913227
Name: vx, dtype: float64

In [126]:
np.sqrt((df_list[0].vx)**2 + (df_list[0].vx)**2).median()

np.float64(9.394620094360661)

In [ ]:
df_list[0][df_list[0]["N"]==4]

,N,Time,xpos,ypos,orientation,fx,fy,torque,vx,vy,ax,ay
23,4,1,4.899037,-8.134722,0.840030,16.757685,-29.252663,-0.0,11.462596,5.849612,671.188931,398.048078
33,4,2,4.937686,-8.014913,0.834345,26.564735,-48.026602,-0.0,3.864846,11.980848,-759.775021,613.123522
43,4,3,4.932785,-8.049020,0.819209,27.963082,-47.936425,-0.0,-0.490099,-3.410662,-435.494437,-1539.150984
53,4,4,5.071301,-8.051299,0.778945,27.692560,-44.126899,-0.0,13.851656,-0.227933,1434.175433,318.272928
63,4,5,5.172870,-8.027156,0.762031,30.525618,-45.784491,-0.0,10.156866,2.414353,-369.479000,264.228631
...,...,...,...,...,...,...,...,...,...,...,...,...
9963,4,995,4.320188,8.874673,0.591974,0.665247,-3.892762,-0.0,5.014731,13.760496,-957.741108,1206.446679
9973,4,996,4.386558,8.913748,0.568634,-2.286769,-9.620841,-0.0,6.636991,3.907509,162.225978,-985.298652
9983,4,997,4.498110,8.971097,0.558096,-12.223369,-32.355956,-0.0,11.155241,5.734939,451.825055,182.742917
9993,4,998,4.612857,9.032322,0.577019,-15.455810,-44.822836,-0.0,11.474686,6.122526,31.944512,38.758780


In [16]:
num_exp = len(df_list)
Np = df_list[0].N.max()
num_steps = len(np.unique(df_list[0].Time))

## Position reshaping

In [9]:
ppos = []
for df in df_list:
    Np = df.N.max()
    x = np.array(df.xpos).reshape(Np,-1)
    y = np.array(df.ypos).reshape(Np,-1)
    xy = np.stack([x,y],axis=2)
    ppos.append(xy)
# position = np.concatenate(ppos,axis=1)
position = np.stack(ppos,axis=3)
position = np.transpose(position,(3,1,0,2))


## Velocity reshaping

In [10]:
vv = []
for df in df_list:
    vx = np.array(df.vx).reshape(Np,-1)
    vy = np.array(df.vy).reshape(Np,-1)
    vxy = np.stack([vx,vy],axis=2)
    vv.append(vxy)
vel = np.stack(vv,axis=3)
vel = np.transpose(vel,(3,1,0,2))

## Force

In [11]:
force_ = []
for df in df_list:
    Np = df.N.max()
    fx = np.array(df.fx).reshape(Np,-1)
    fy = np.array(df.fy).reshape(Np,-1)
    fxy = np.stack([fx,fy],axis=2)
    force_.append(fxy)
# position = np.concatenate(ppos,axis=1)
force = np.stack(force_,axis=3)
force = np.transpose(force,(3,1,0,2))

## Orientation

In [65]:
angle_ = []
for df in df_list:
    Np = df.N.max()
    theta = np.array(df.orientation).reshape(Np,-1)
    angle_.append(theta)
angle = np.stack(angle_, axis=2)[:,np.newaxis]
angle = np.transpose(angle,(3,2,0,1))

## Drag Coefficient

In [18]:
R = 2.
gamma = 6*np.pi*R*1e-3
gamma = np.repeat(gamma, Np)
gamma = gamma[:,np.newaxis]
gamma = np.tile(gamma,(num_exp,num_steps,1,1))

## Edges

In [25]:
edges = list(itertools.combinations(range(Np), 2))
edges = np.array(edges)

In [26]:
data = {
    "position": position,
    "velocity": vel,
    "drag_coefficient": gamma,
    "edge_list": edges,
}

In [27]:
with open('data.pkl', '+rb') as f:
    pkl.dump(data, f)